# Model Free Analysis

Statistical inquiry into the aggregate behaviour of the *Wisconsin Sorting* & *NBack* Tasks.


---------
```
Zach Wolpe
zachcolinwolpe@gmail.com
27 July 2021
```
---------



# Executive Functions

The additional experiments are provided to gauge executive functions and computer literacy that may distinguish candidates when participating in the WCST & NBack Tasks.

In [38]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os
import re
import sys
sys.path.append('../process data/')
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.express as px
from encode_processed_data import encode_data

In [2]:
# with open('../data objects/batch_processing_object.pkl', 'rb') as file2:
#     bp = pickle.load(file2)

# ---- fetch data object ----x
with open('../data objects/batch_processing_object_with_encodings.pkl', 'rb') as file2:
    ed = pickle.load(file2)
ed.__dict__.keys()

dict_keys(['raw', 'fitts_summary_stats', 'corsi_summary_stats', 'navon_summary_stats', 'nback_summary_stats', 'demographics'])

In [3]:
ed.describe_data()



        ------------------------------------------------------------------
            self.path            : raw data loc
            self.metadata        : mturk metadata
            self.mapping         : reference table
            self.data_times      : reference times table
            self.participants    : list of participant identifiers
            self.parti_code      : list of participant codes
            self.n               : total number of samples
            self.wcst_paths      : paths to wcst  raw data
            self.nback_paths     : paths to nback raw data
            self.corsi_paths     : paths to corsi raw data
            self.fitts_paths     : paths to fitts raw data
            self.navon_paths     : paths to navon raw data
            self.wcst_data       : wcst  dataframe
            self.nback_data      : nback dataframe
            self.corsi_data      : corsi dataframe
            self.fitts_data      : fitts dataframe
            self.navon_data    

In [4]:
ed.nback_summary_stats.head()

participant block_number trial_counter score              status            \
                                   count  mean       std      mean       std   
0    100934.0            1            20  0.30  0.470162  0.850000  0.366348   
1    100934.0            2            30  0.30  0.466092  0.900000  0.305129   
2    100934.0            3            30  0.20  0.406838  0.933333  0.253708   
3    103322.0            1            20  0.25  0.444262  0.850000  0.366348   
4    103322.0            2            30  0.30  0.466092  0.933333  0.253708   

       miss           false_alarm           reaction_time_ms               
       mean       std        mean       std             mean          std  
0  0.250000  0.444262    0.050000  0.223607      2155.400000  1185.698080  
1  0.266667  0.449776    0.033333  0.182574      2175.133333  1191.591626  
2  0.200000  0.406838    0.000000  0.000000      2323.700000  1142.043364  
3  0.200000  0.410391    0.050000  0.223607      2305.100000  1090.350542  
4  0.233333  0.430183    0.066667  0.253708      2425.966667  1059.981668

In [5]:
ed.navon_summary_stats.head()

participant level_of_target   correct           too_slow       \
                                   mean       std     mean  std   
0    100934.0          global  0.166667  0.389249      0.0  0.0   
1    100934.0           local  0.461538  0.518875      0.0  0.0   
2    100934.0            none  0.480000  0.509902      0.0  0.0   
3    103322.0          global  0.923077  0.277350      0.0  0.0   
4    103322.0           local  0.916667  0.288675      0.0  0.0   

  reaction_time_ms              
              mean         std  
0       664.583333  221.989540  
1       597.769231  294.353856  
2       622.960000  343.110949  
3       959.769231  358.604739  
4       999.750000  221.561453

In [12]:
ed.corsi_summary_stats.head()

participant highest_span n_items    status          
                       max     max      mean       std
0    100934.0            4       5  0.428571  0.534522
1    103322.0            6       7  0.625000  0.517549
2    107700.0            6       7  0.625000  0.517549
3    117200.0            4       5  0.500000  0.547723
4    117306.0            5       6  0.571429  0.534522

In [13]:
ed.fitts_summary_stats.head()

participant   delta             status          
                 mean         std   mean       std
0    100934.0   -5.70  186.607130    1.0  0.000000
1    103322.0 -203.05  174.308400    1.0  0.000000
2    107700.0 -342.65  345.637059    1.1  0.447214
3    117200.0  -85.05  156.117188    1.0  0.000000
4    117306.0 -141.80  167.808036    1.0  0.000000

In [14]:
ed.demographics.iloc[0,]

participant                                             816404.0
participant_file      s.6463d918-7e29-49b0-942d-f1a9faab6ad6.txt
user_agent                                           mozilla/5.0
Welcome_Screen_T                                 1621951320945.0
participant_code_a                                      816404.0
feedback_T                                                8719.0
age_T                                                     3556.0
age_a                                                       28.0
gender_T                                                  2023.0
gender_a                                                  female
handedness_T                                              3627.0
handedness_a                                               right
education_T                                               2604.0
education_a                                           university
income_T                                                  6457.0
income_a                 

# Final Datasets

These curated datasets are now joined, pruned & transformed to produce the `final` set of datasets - on which we will conduct our anaylsis.


In [72]:
# --- demographics dataset ---x
ed.demographics.columns

df2 = ed.demographics[[ 'participant', 'age_a','gender_a','handedness_a','education_a', 'income_a', 
                        'computer_hours_a','age_group']]

ed.raw.wcst_data[['participant', 'seq_no', 'card_no', 'rule', 'card_shape', 'card_number',
                        'card_colour', 'reaction_time_ms',
                        'status', 'card_selected', 'error','perseverance_error', 'not_perseverance_error']]

['participant', 'participant_code', 'card_no', 'correct_card',
       'correct_persevering', 'seq_no', 'rule', 'card_shape', 'card_number',
       'card_colour', 'reaction_time_ms', 'status', 'card_selected', 'error',
       'perseverance_error', 'not_perseverance_error'],

       
ed.raw.wcst_data 
                    

,participant,participant_code,card_no,correct_card,correct_persevering,seq_no,rule,card_shape,card_number,card_colour,reaction_time_ms,status,card_selected,error,perseverance_error,not_perseverance_error
0,816404.0,s.6463d918-7e29-49b0-942d-f1a9faab6ad6.txt,52,1,0,1,shape,triangle,1,yellow,3775,1,1,0,0,0
1,816404.0,s.6463d918-7e29-49b0-942d-f1a9faab6ad6.txt,59,3,0,2,shape,triangle,3,red,3771,1,3,0,0,0
2,816404.0,s.6463d918-7e29-49b0-942d-f1a9faab6ad6.txt,23,2,0,3,shape,cross,2,red,1359,1,2,0,0,0
3,816404.0,s.6463d918-7e29-49b0-942d-f1a9faab6ad6.txt,52,1,0,4,shape,triangle,1,yellow,1209,1,1,0,0,0
4,816404.0,s.6463d918-7e29-49b0-942d-f1a9faab6ad6.txt,52,1,0,5,shape,triangle,1,yellow,953,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27395,684712.0,s.7eeb88a1-4136-4dad-923e-c98785b34377.txt,61,4,2,2,color,triangle,4,blue,1761,1,4,0,0,0
27396,684712.0,s.7eeb88a1-4136-4dad-923e-c98785b34377.txt,14,4,1,3,color,circle,4,green,1109,1,4,0,0,0
27397,684712.0,s.7eeb88a1-4136-4dad-923e-c98785b34377.txt,14,4,1,4,color,circle,4,green,1045,1,4,0,0,0
27398,684712.0,s.7eeb88a1-4136-4dad-923e-c98785b34377.txt,23,2,3,5,color,cross,2,red,1779,1,2,0,0,0


In [61]:
# ---- Scatter Plots ----x

# --- data ---x
df1 = ed.raw.wcst_data[['participant', 'seq_no', 'card_no', 'rule', 'card_shape', 'card_number',
                        'card_colour', 'reaction_time_ms',
                        'status', 'card_selected', 'error','perseverance_error', 'not_perseverance_error']]
df2 = ed.demographics[[ 'participant', 'age_a','gender_a','handedness_a','education_a', 'income_a', 
                        'computer_hours_a','age_group']]
df  = df1.set_index('participant').join(df2.set_index('participant'))


# --- scatter plot ---x
trace_1 = go.Scatter(x=df.computer_hours_a, y=df.age_a, mode='markers', marker=dict(size=12, color='steelblue', symbol='pentagon', line={'width':2}))
data    = [trace_1]
layout  = go.Layout(title='title', xaxis={'title':'age'}, yaxis={'title':'computer hours'}, hovermode='closest')
fig     = go.Figure(data=data, layout=layout)
pyo.plot(fig, filename='temp-plot.html')

'temp-plot.html'

In [66]:
# ---- line plot ----x
trace_1 = go.Scatter(x=df.seq_no, y=df.gender_a, mode='markers', marker=dict(size=5, color='darkblue', symbol='circle'))
trace_2 = go.Scatter(x=df.seq_no, y=df.education_a, mode='markers', marker=dict(size=5, color='darkblue', symbol='circle'))

data    = [trace_1, trace_2]
layout  = go.Layout(title='R Time', xaxis={'title':'age'}, yaxis={'title':'computer hours'})
fig = go.Figure(data=data, layout=layout)
pyo.plot(fig)


'temp-plot.html'